<a href="https://colab.research.google.com/github/ytg000629/daegu_bigdata/blob/main/KoBERT_%EB%89%B4%EC%8A%A4%EC%B9%B4%ED%85%8C%EA%B3%A0%EB%A6%AC%EB%B6%84%EB%A5%98ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# install 및 import

In [2]:
#!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

In [3]:
# ★
!pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'

  Cloning https://github.com/SKTBrain/KoBERT.git to /tmp/pip-install-6rd654gh/kobert-tokenizer_642c14aa9b9a42898a66b0304ad6f882
  Running command git clone --filter=blob:none --quiet https://github.com/SKTBrain/KoBERT.git /tmp/pip-install-6rd654gh/kobert-tokenizer_642c14aa9b9a42898a66b0304ad6f882
  Resolved https://github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done
  Created wheel for kobert_tokenizer: filename=kobert_tokenizer-0.1-py3-none-any.whl size=4632 sha256=2e64efd39e66e922a37607fb494a53b72157f9c837d116a0d6274f72b17b9f3e
  Stored in directory: /tmp/pip-ephem-wheel-cache-774b7x3b/wheels/e9/1a/3f/a864970e8a169c176befa3c4a1e07aa612f69195907a4045fe
Successfully built kobert_tokenizer


In [4]:
!pip install mxnet
!pip install gluonnlp==0.8.0
!pip install sentencepiece==0.1.96
!pip install transformers
!pip install torch

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 17.6 MB/s eta 0:00:00
  Attempting uninstall: graphviz
    Found existing installation: graphviz 0.20.1
    Uninstalling graphviz-0.20.1:
      Successfully uninstalled graphviz-0.20.1
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 235.5/235.5 kB 4.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gluonnlp: filename=gluonnlp-0.8.0-py3-none-any.whl size=292697 sha256=2b5dec76f38275709b8af6f9ef5d902a04f8b6b3d526c73ff716730c26024ade
  Stored in directory: /root/.cache/pip/wheels/2d/cc/dc/7ec84dced25f738b8be400101abb67e4b50c905090a51017e4
Successfully built gluonnlp
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 63.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 34.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 110.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━

In [5]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm
from tqdm import notebook
from sklearn.model_selection import train_test_split
import re
import string
import pandas as pd

/usr/local/lib/python3.10/dist-packages/mxnet/optimizer/optimizer.py:163: UserWarning: WARNING: New optimizer gluonnlp.optimizer.lamb.LAMB is overriding existing optimizer mxnet.optimizer.optimizer.LAMB
  warnings.warn('WARNING: New optimizer %s.%s is overriding '


In [6]:
## GPU
device = torch.device("cuda:0")

In [7]:
#Hugging Face를 통한 모델 및 토크나이저 Import
from transformers import BertModel
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup
from kobert_tokenizer import KoBERTTokenizer

In [8]:
tokenizer = KoBERTTokenizer.from_pretrained('skt/kobert-base-v1')
bertmodel = BertModel.from_pretrained('skt/kobert-base-v1', return_dict=False)
vocab = nlp.vocab.BERTVocab.from_sentencepiece(tokenizer.vocab_file, padding_token='[PAD]')

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


# 데이터 준비

In [78]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [79]:
one = pd.read_csv('/content/drive/MyDrive/대구 빅데이터/data/학습용_데이터/생활0.csv')
two = pd.read_csv('/content/drive/MyDrive/대구 빅데이터/data/학습용_데이터/IT_전자1.csv')
three = pd.read_csv('/content/drive/MyDrive/대구 빅데이터/data/학습용_데이터/의류_패션잡화2.csv')
four = pd.read_csv('/content/drive/MyDrive/대구 빅데이터/data/학습용_데이터/음식3.csv')
five = pd.read_csv('/content/drive/MyDrive/대구 빅데이터/data/학습용_데이터/문화4.csv')
six = pd.read_csv('/content/drive/MyDrive/대구 빅데이터/data/학습용_데이터/교육5.csv')

df = [one, two, three, four, five, six]
df = pd.concat(df, ignore_index=True)

In [80]:
df.head()

,company,url,title,content,label
0,머니S,https://n.news.naver.com/mnews/article/417/000...,'중국 앓이' 대기업 울고 '실속 챙긴' 중소기업 웃고… 희비 갈린 K-뷰티,"[머니S리포트 - 판 커지는 K-뷰티(1)] ①'입소문' 난 중소 브랜드, K-뷰...",0
1,뉴시스,https://n.news.naver.com/mnews/article/003/001...,'최대 66% 할인' 명품페스타…루트에너지 '태양광 펀드'[중기소식],[서울=뉴시스] 대명아임레디 '명품 기획전'. (이미지=대명스테이션 제공) 202...,0
2,스포츠서울,https://n.news.naver.com/mnews/article/468/000...,"성남산업진흥원, 남다른상점 ‘성료’... 11월 또 개장",중소기업 유망브랜드 마켓 ‘남다른상점’. 제공=성남산업진흥원〔스포츠서울│성남=좌승...,0
3,한국경제,https://n.news.naver.com/mnews/article/015/000...,지금은 우리가 사모펀드처럼 투자할 때[김태엽의 PEF썰전],[한경 CFO Insight]김태엽 어펄마캐피탈 한국대표 taeyub.kim@af...,0
4,동아일보,https://n.news.naver.com/mnews/article/020/000...,“살려 달라”…대낮 서울 신림동 공원서 여성 때리고 성폭행,서울 관악구 신림동의 도심 공원에서 여성을 무차별적으로 폭행한 후 성폭행까지 한 ...,0


# 텍스트 데이터 전처리

In [16]:
#불용어 처리

#불용어 사전 구축 후 사용
with open('/content/drive/MyDrive/대구 빅데이터/data/korean_stopwords.txt','r',encoding='utf-8-sig') as f:
    stopwords_list=[]
    example =f.readlines()
    for line in example:
        stopwords_list.append(line.strip())

#추가하고 싶은 불용어가 있다면 다음과 같이 넣어서 사용
korean_stopwords='단독 기자 속보 사진'
stop_words = korean_stopwords.split(' ')

#불용어 사전 정의
stop = stop_words+stopwords_list

In [81]:
#중복 기사 제거

print(f'중복 제거 전 train length: {len(df)}')
df.drop_duplicates(subset=['content'], inplace = True, ignore_index = True)
print(f'중복 제거 후 train length: {len(df)}\n')

중복 제거 전 train length: 2906
중복 제거 후 train length: 2890



In [82]:
#전처리를 위한 함수

def text_preproccessing(df):

    df = df.copy()


    # HTML tag 제거
    def remove_html(text):
        html=re.compile(r'<.*?>')
        return html.sub(r'',text)


    # 구두점 제거
    def remove_punct(text):
        table=str.maketrans('','',string.punctuation)
        return text.translate(table)


    # 불용어 제거
    def remove_stopwords(text):
        return ' '.join([i for i in text.split() if i not in stop])


    # 특수문자,영어,숫자 제거(한글과 공백문자 제외)
    def remove_nokorean(text):
        korean = re.sub('[^가-힣\s]', '', text)
        return korean


    # 2개 이상의 연속된 공백을 하나의 공백으로 대체
    def replace_space(text):
        space = re.sub(' {2,}', ' ', text)
        return space


    # 줄바꿈 문자를 하나의 공백으로 대체
    def replace_enter(text):
        replaced_text = re.sub('\n', ' ', text)
        return replaced_text


    df['content']=df['content'].apply(lambda x : remove_html(x))
    df['content']=df['content'].apply(lambda x : remove_punct(x))
    df['content']=df['content'].apply(lambda x : remove_stopwords(x))
    df['content']=df['content'].apply(lambda x : remove_nokorean(x))
    df['content']=df['content'].apply(lambda x : replace_space(x))
    df['content']=df['content'].apply(lambda x : replace_enter(x))


    return df

In [83]:
df = text_preproccessing(df)

In [84]:
df.head()

,company,url,title,content,label
0,머니S,https://n.news.naver.com/mnews/article/417/000...,'중국 앓이' 대기업 울고 '실속 챙긴' 중소기업 웃고… 희비 갈린 K-뷰티,머니리포트 판 커지는 뷰티 입소문 난 중소 브랜드 뷰티 아쉬움 채워편집자주아모레퍼시...,0
1,뉴시스,https://n.news.naver.com/mnews/article/003/001...,'최대 66% 할인' 명품페스타…루트에너지 '태양광 펀드'[중기소식],서울뉴시스 대명아임레디 명품 기획전 이미지대명스테이션 제공 서울뉴시스 배민욱 일 중...,0
2,스포츠서울,https://n.news.naver.com/mnews/article/468/000...,"성남산업진흥원, 남다른상점 ‘성료’... 11월 또 개장",중소기업 유망브랜드 마켓 남다른상점 제공성남산업진흥원스포츠서울성남좌승훈기자성남산업진...,0
3,한국경제,https://n.news.naver.com/mnews/article/015/000...,지금은 우리가 사모펀드처럼 투자할 때[김태엽의 PEF썰전],한경 김태엽 어펄마캐피탈 한국대표 이 기사는 월 일 마켓인사이트에 게재된 기사입니다...,0
4,동아일보,https://n.news.naver.com/mnews/article/020/000...,“살려 달라”…대낮 서울 신림동 공원서 여성 때리고 성폭행,서울 관악구 신림동의 도심 공원에서 여성을 무차별적으로 폭행한 후 성폭행까지 남성이...,0


In [24]:
#okt = Okt()
#max_length = df['content'].apply(lambda x: len(okt.morphs(x))).max()
#max_length

#9141

In [25]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, vocab, max_len,
                 pad, pair):
        transform = BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len,vocab=vocab, pad=pad, pair=pair)
        #transform = nlp.data.BERTSentenceTransform(
        #    tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))



In [26]:
class BERTSentenceTransform:
    r"""BERT style data transformation.

    Parameters
    ----------
    tokenizer : BERTTokenizer.
        Tokenizer for the sentences.
    max_seq_length : int.
        Maximum sequence length of the sentences.
    pad : bool, default True
        Whether to pad the sentences to maximum length.
    pair : bool, default True
        Whether to transform sentences or sentence pairs.
    """

    def __init__(self, tokenizer, max_seq_length,vocab, pad=True, pair=True):
        self._tokenizer = tokenizer
        self._max_seq_length = max_seq_length
        self._pad = pad
        self._pair = pair
        self._vocab = vocab

    def __call__(self, line):
        """Perform transformation for sequence pairs or single sequences.

        The transformation is processed in the following steps:
        - tokenize the input sequences
        - insert [CLS], [SEP] as necessary
        - generate type ids to indicate whether a token belongs to the first
        sequence or the second sequence.
        - generate valid length

        For sequence pairs, the input is a tuple of 2 strings:
        text_a, text_b.

        Inputs:
            text_a: 'is this jacksonville ?'
            text_b: 'no it is not'
        Tokenization:
            text_a: 'is this jack ##son ##ville ?'
            text_b: 'no it is not .'
        Processed:
            tokens: '[CLS] is this jack ##son ##ville ? [SEP] no it is not . [SEP]'
            type_ids: 0     0  0    0    0     0       0 0     1  1  1  1   1 1
            valid_length: 14

        For single sequences, the input is a tuple of single string:
        text_a.

        Inputs:
            text_a: 'the dog is hairy .'
        Tokenization:
            text_a: 'the dog is hairy .'
        Processed:
            text_a: '[CLS] the dog is hairy . [SEP]'
            type_ids: 0     0   0   0  0     0 0
            valid_length: 7

        Parameters
        ----------
        line: tuple of str
            Input strings. For sequence pairs, the input is a tuple of 2 strings:
            (text_a, text_b). For single sequences, the input is a tuple of single
            string: (text_a,).

        Returns
        -------
        np.array: input token ids in 'int32', shape (batch_size, seq_length)
        np.array: valid length in 'int32', shape (batch_size,)
        np.array: input token type ids in 'int32', shape (batch_size, seq_length)

        """

        # convert to unicode
        text_a = line[0]
        if self._pair:
            assert len(line) == 2
            text_b = line[1]

        tokens_a = self._tokenizer.tokenize(text_a)
        tokens_b = None

        if self._pair:
            tokens_b = self._tokenizer(text_b)

        if tokens_b:
            # Modifies `tokens_a` and `tokens_b` in place so that the total
            # length is less than the specified length.
            # Account for [CLS], [SEP], [SEP] with "- 3"
            self._truncate_seq_pair(tokens_a, tokens_b,
                                    self._max_seq_length - 3)
        else:
            # Account for [CLS] and [SEP] with "- 2"
            if len(tokens_a) > self._max_seq_length - 2:
                tokens_a = tokens_a[0:(self._max_seq_length - 2)]

        # The embedding vectors for `type=0` and `type=1` were learned during
        # pre-training and are added to the wordpiece embedding vector
        # (and position vector). This is not *strictly* necessary since
        # the [SEP] token unambiguously separates the sequences, but it makes
        # it easier for the model to learn the concept of sequences.

        # For classification tasks, the first vector (corresponding to [CLS]) is
        # used as as the "sentence vector". Note that this only makes sense because
        # the entire model is fine-tuned.
        #vocab = self._tokenizer.vocab
        vocab = self._vocab
        tokens = []
        tokens.append(vocab.cls_token)
        tokens.extend(tokens_a)
        tokens.append(vocab.sep_token)
        segment_ids = [0] * len(tokens)

        if tokens_b:
            tokens.extend(tokens_b)
            tokens.append(vocab.sep_token)
            segment_ids.extend([1] * (len(tokens) - len(segment_ids)))

        input_ids = self._tokenizer.convert_tokens_to_ids(tokens)


        # The valid length of sentences. Only real  tokens are attended to.
        valid_length = len(input_ids)

        if self._pad:
            # Zero-pad up to the sequence length.
            padding_length = self._max_seq_length - valid_length
            # use padding tokens for the rest
            input_ids.extend([vocab[vocab.padding_token]] * padding_length)
            segment_ids.extend([0] * padding_length)

        return np.array(input_ids, dtype='int32'), np.array(valid_length, dtype='int32'),\
            np.array(segment_ids, dtype='int32')




In [27]:
train_data, val_data, train_label, val_label = train_test_split(df['content'].values,
                                                df['label'].values,
                                                test_size=0.2,
                                                stratify=df['label'].values,
                                                random_state=10)

In [28]:
train_label = list(map(str, train_label))
val_label = list(map(str, val_label))

In [29]:
dataset_train = [[data, label] for data, label in zip(train_data, train_label)]
dataset_test = [[data, label] for data, label in zip(val_data, val_label)]

In [30]:
dataset_train[:2]

[['제주 이민 년차들을 만나다 효율과 자율의 삶을 추구하는 지준호씨 제주 이주 열풍 이 분 어느덧 년이 넘었다 지난 년 제주를 떠난 사람도 있지만 남아있는 사람도 많다 그들의 진짜 삶이 궁금해 직접 인터뷰에 나섰다 오십 대 남자가 산다라고 이미지가 떠오를까 흐트러진 일상과 궁상맞아 보이는 집구석 굴러 다니는 술병 편견이 무섭다제주도 십 년차 이주민 인터뷰를 기획하면서 가장 찾기 어려운 경우가 십 제주에서 홀로 버틴 남자였다 홀로 사는 이주민들은 주위에 정말 많았다 남자는 드물었다 한국 사회에서 남자는 혼자서 살기 어려운 존재인 걸까사십 대 초반 혈혈단신 제주에 내려온 뒤 십 년째 홀로 살고 있는 남자분이 있어 인터뷰를 요청했다 제주 동쪽 세화리에서 민박집 탱자싸롱을 운영 중인 지준호 씨가 주인공이다제주 이주 아이를 낳아 기른다고 정신없이 시간을 보낸 나와는 달리 지준호씨는 선연하게 시간의 결을 느끼며 십 년을 살았을 같다는 생각이 들었다 그에게 제주에서의 십 년은 시간이었을까 자유롭게 살고 싶어서 프랑스 마르세유에서 지준호씨 유럽 여행 프랑스 마르세유 앞바다 부근에서 지준호씨가 휴식을 취하고 지준호 십 제주에 오셨지가 궁금해요그때 제주가 한창 핫할 때였어요 이주민들이 많아지고 있었죠 제주는 있는 걸 못 사람은 맞는다는데 저는 놀거든요 가볼까 생각이 들었죠 년 광고 쪽 일을 했는데 계속 나갈지도 의문이고 새로운 게 싶기도 자유롭게 살고 싶다는 생각도 들었어요 왠지 가면 좋은 여자를 만날 예감도 들었고요웃음 휴가 내고 왔는데 너무 좋아서 달 만에 사표를 내고 왔어요 기회가 닿아 땅을 사게 됐어요 게스트하우스 걸 해보고 싶다는 마음은 늘 있었거든요 사십 대 아직 젊을 게스트하우스를 열어서 손님들이랑 놀고 싶다는 생각이 들었어요 처음 년은 정말 재미있게 지냈어요손님들 왠지 이야기가 통할 분들이 있거든요 제가 지적인 대화를 좋아해서 관심사가 비슷한 분들을 만나면 이야기도 많이 술잔도 기울이고 그랬죠 제가 정말 진심으로 손님들한테 다가가고 소통을 하니까 즐거웠어요 그때

In [31]:
print(len(dataset_train))
print(len(dataset_test))

2324
582


In [32]:
# Setting parameters
max_len = 512
batch_size = 4
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [33]:
# ★


data_train = BERTDataset(dataset_train, 0, 1, tokenizer,vocab, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tokenizer, vocab, max_len, True, False)

In [34]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=2) #5
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=2)   #5

In [35]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,          ############
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate

        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)

    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)

        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [36]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [37]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [38]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [39]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [40]:
#batch size 크기를 줄여가면서 gpu 캐시를 비워주는 코드

import gc
import torch

gc.collect()
torch.cuda.empty_cache()

In [41]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [42]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 1/581 [00:02<19:27,  2.01s/it]

epoch 1 batch id 1 loss 1.9637162685394287 train acc 0.25


 35%|███▍      | 201/581 [01:22<02:43,  2.33it/s]

epoch 1 batch id 201 loss 0.5682252645492554 train acc 0.43034825870646765


 69%|██████▉   | 401/581 [02:47<01:17,  2.33it/s]

...

epoch 5 batch id 401 loss 0.7308350801467896 train acc 0.949501246882793


100%|██████████| 581/581 [04:06<00:00,  2.35it/s]


epoch 5 train acc 0.9565404475043029


100%|██████████| 146/146 [00:21<00:00,  6.93it/s]

epoch 5 test acc 0.8904109589041096


In [1]:
## 학습 모델 저장
PATH = '/content/drive/MyDrive/대구 빅데이터/'
torch.save(model, PATH + 'koert_train.pkl')                                     # 전체 모델 저장
torch.save(model.state_dict(), PATH + 'model_state_dict.pkl')               # 모델 객체의 state_dict 저장
torch.save({
    'model': model.state_dict(),
    'optimizer': optimizer.state_dict()
}, PATH + 'all.tar')  # 여러 가지 값 저장, 학습 중 진행 상황 저장을 위해 epoch, loss 값 등 일반 scalar값 저장 가능

NameError: ignored

In [62]:
## 학습 모델 로드
PATH = '/content/drive/MyDrive/대구 빅데이터/'
model = torch.load(PATH + 'koert_train.pkl')  # 전체 모델을 통째로 불러옴, 클래스 선언 필수
model.load_state_dict(torch.load(PATH + 'model_state_dict.pkl'))  # state_dict를 불러 온 후, 모델에 저장


def new_softmax(a) :
    c = np.max(a) # 최댓값
    exp_a = np.exp(a-c) # 각각의 원소에 최댓값을 뺀 값에 exp를 취한다. (이를 통해 overflow 방지)
    sum_exp_a = np.sum(exp_a)
    y = (exp_a / sum_exp_a) * 100
    return np.round(y, 3)


# 예측 모델 설정
def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tokenizer, vocab, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=2)

    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)

        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()
            min_v = min(logits)
            total = 0
            category = ['생활', 'IT_전자', '의류_패션잡화', '음식', '문화', '교육']
            probability = []
            logits = np.round(new_softmax(logits), 3).tolist()
            for logit in logits:
                probability.append(np.round(logit, 3))
                sort_prob = sorted(probability, reverse=True)


            answer = []



            for i in range(len(probability)):
                for j in range(1):
                    if probability[i] == sort_prob[j]:
                        answer.append(category[i])


    return answer,  probability

In [72]:
predict('''튀르키예의 곡물 등 식량 가격이 치솟아 소비자들이 긴장과 우려 속에 살고 있다고 중국 관영 신화이 20일(현지시간) 보도했다.

여름철은 튀르키예에서는 국민들이 보통 치솟는 물가 걱정을 잠시 내려놓는 유예기간이 되어왔지만 올 여름은 그렇지가 않다고 보도했다.

기후변화로 인한 농산물 생산량의 감소에다 높은 인플레이션 때문에 식량과 식품 가격이 계속 치솟아 튀르키예 소비자들이 극심한 긴장을 느끼고 있다는 것이다.

튀르키예 통계청이 발표한 7월 인플레 통계에 따르면 인플레이션 상승 비율은 전년도에 비해 거의 50%나 되었다.  가장 가격이 많이 오른 농식품으로는 채소가 20.1%로 선두를 달리고 있고 신선한 과일류가 15.7%로 그 다음이었다.

수도 앙카라의 100개 지역 중 한 시장에서는 이런 물가 때문에 소비자와 판매자들이 모두 불만에 가득차 있었다.

20대 남성 고객인 에제 오즈칸은 신화통신 기자에게 시장의 모든 품목이 전 주와 그 전 주에 비해 계속 올라서 가격표가 매번 새로 바뀌고 있다고 말했다.

그 때문에 식료품을 사는 것에서부터 사회활동 전반까지 모든 지출을 최소로 줄일 수 밖에 없다고 그는 불평했다.

수박을 판매하는 에르도안 아타스도 "우리 상인들도 물가가 너무 올라서 손님들이 우리 물건을 사기 힘들게 돼버리는 바람에 판매하기가 힘들다"고 하소연했다.  농사에 드는 비용까지 오르는 바람에 수박값도 이미 100~130%가 인상되었다는 것이다.''')

(['IT_전자'], [0.029, 99.812, 0.027, 0.051, 0.067, 0.015])

In [73]:
import pandas as pd

# 데이터프레임 생성 (예시 데이터)
data = {'고객 ID': [1, 1, 2, 3, 2],
        '승인 금액': [100, 200, 150, 300, 120],
        '업종 카테고리': ['A', 'B', 'A', 'C', 'B']}
df = pd.DataFrame(data)

# 고객별로 그룹화하여 승인 금액과 건수 합계 계산
grouped = df.groupby(['고객 ID', '업종 카테고리']).agg({'승인 금액': 'sum', '업종 카테고리': 'count'})

# 그룹별 승인 금액과 건수 비율 계산
grouped['금액 비율'] = grouped['승인 금액'] / grouped.groupby('고객 ID')['승인 금액'].transform('sum')
grouped['건수 비율'] = grouped['업종 카테고리'] / grouped.groupby('고객 ID')['업종 카테고리'].transform('sum')

# 결과 출력
print(grouped)

               승인 금액  업종 카테고리     금액 비율  건수 비율
고객 ID 업종 카테고리                                 
1     A          100        1  0.333333    0.5
      B          200        1  0.666667    0.5
2     A          150        1  0.555556    0.5
      B          120        1  0.444444    0.5
3     C          300        1  1.000000    1.0


In [74]:
import pandas as pd
import numpy as np

# 데이터프레임 생성 (예시 데이터)
data = {'고객 ID': [1, 1, 2, 3, 2],
        '승인 금액': [100, 200, 150, 300, 120],
        '업종 카테고리': ['A', 'B', 'A', 'C', 'B']}
df = pd.DataFrame(data)

# 고객별 업종 개수와 업종 목록 가져오기
customer_categories = df.groupby('고객 ID')['업종 카테고리'].unique()

# 고객별 선호도 벡터 생성
preference_vectors = []
for categories in customer_categories:
    customer_data = df[df['업종 카테고리'].isin(categories)]
    total_approval_amount = customer_data['승인 금액'].sum()
    preference_vector = np.zeros(len(categories))
    for index, row in customer_data.iterrows():
        category_index = np.where(categories == row['업종 카테고리'])[0][0]
        preference_vector[category_index] = row['승인 금액'] / total_approval_amount
    preference_vectors.append(preference_vector)

# 결과 출력
for i, vector in enumerate(preference_vectors):
    print(f"고객 {i+1}의 선호도 벡터: {vector}")

고객 1의 선호도 벡터: [0.26315789 0.21052632]
고객 2의 선호도 벡터: [0.26315789 0.21052632]
고객 3의 선호도 벡터: [1.]
